In [1]:
from collections import Counter
from collections import OrderedDict
import numpy as np
import pandas
import heapq

In [2]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('train.txt', 'r') as f:
    for line in f.xreadlines():
        visits, purchases = line.strip().split(';')
        for visit in visits.split(','):
            visit_popularity[visit] += 1
        if purchases != '':
            for purchase in purchases.split(','):
                purchase_popularity[purchase] += 1

In [3]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_visit(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: visit_popularity.get(x, 0))

In [34]:
def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        avg_precision = np.zeros(max_count)
        sessions_count = 0
        for line in f.xreadlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = set(purchases.split(','))
                rec = recommend(visits, max_count)
                rec_hits = np.array(map(lambda x: x in purchases, rec))
                hits = np.zeros(max_count)
                hits[:len(rec_hits)] = rec_hits
                sessions_count += 1
                avg_recall +=  np.cumsum(hits) / len(purchases)
                avg_precision += np.cumsum(hits) / (np.arange(max_count) + 1)

    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / sessions_count],
            'avg_precision@k': [round(x, 2) for x in avg_precision / sessions_count]
    }).set_index('k')


In [36]:
calculate_metrics(recommend_by_purchase, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.80,0.69
2,0.53,0.84
3,0.39,0.89
4,0.31,0.91
5,0.25,0.93


In [37]:
calculate_metrics(recommend_by_purchase, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.53,0.46
2,0.38,0.64
3,0.30,0.73
4,0.25,0.79
5,0.21,0.82


In [38]:
calculate_metrics(recommend_by_visit, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.51,0.44
2,0.38,0.63
3,0.30,0.73
4,0.25,0.79
5,0.21,0.82


In [39]:
calculate_metrics(recommend_by_visit, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.48,0.42
2,0.36,0.60
3,0.29,0.70
4,0.24,0.76
5,0.20,0.80


# Описание задания

##Входные данные

Имеются файл train.txt и test.txt

В них записаны сессии по одной в каждой строке.

Формат сессии такой: id просмотренных товаров через "," затем идёт ";" после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, "1,2,3,4;" или "1,2,3,4;5,6".

Гарантируется, что среди id купленных товаров все различные.

##Что нужно сделать
1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появлять в просмотренных, это надо учитывать)
2. Реализуйте два алгоритма рекомендации: сортировка просмотренных id по популярности (частота появления в просмотренных), сортировка просморенных id по покупаемости (частота появления в покупках). Если частота одинаковая, то сортировать надо по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)
3. Для данных алгоритмов найдите **AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5**. На обучющей и тестовых выборках, округляя до 2 знака после запятой.


##Ожидаемые ответы
Популярность на обучающей:
>**AverageRecall@1: 0.44, AveragePrecision@1: 0.51, AverageRecall@5: 0.82, AveragePrecision@5: 0.21**

Популярность на тестовой
>**AverageRecall@1: 0.42, AveragePrecision@1: 0.48, AverageRecall@5: 0.80, AveragePrecision@5: 0.20**

Покупаемость на обучающей
>**AverageRecall@1: 0.69, AveragePrecision@1: 0.80, AverageRecall@5: 0.93, AveragePrecision@5: 0.25**

Покупаемость на тестовой
>**AverageRecall@1: 0.46, AveragePrecision@1: 0.53, AverageRecall@5: 0.82, AveragePrecision@5: 0.21**